## The Return Story

In [2]:
import pandas as pd
import plotly.express as px
from collections import Counter
import numpy as np

## Dataset Understanding

In [3]:
df = pd.read_excel('online-retail-assessment-data.xlsx')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,491639,20668,NaN,-1395,2009-12-11 16:02:00,0.0,NaN,United Kingdom
1,489660,35956,lost,-1043,2009-12-01 17:43:00,0.0,NaN,United Kingdom
2,490016,21982,NaN,-1012,2009-12-03 12:30:00,0.0,NaN,United Kingdom
3,489899,79323GR,sold as gold,-954,2009-12-03 09:41:00,0.0,NaN,United Kingdom
4,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom


In [4]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,32198.000000,32198,32198.000000,23838.000000
mean,9.129356,2009-12-07 16:09:20.876451840,4.150301,15467.566952
min,-1395.000000,2009-12-01 07:45:00,0.000000,12346.000000
25%,1.000000,2009-12-03 14:39:00,1.250000,14085.000000
50%,3.000000,2009-12-07 16:26:00,2.570000,15542.000000
75%,8.000000,2009-12-10 16:39:00,4.950000,16905.000000
max,5184.000000,2009-12-15 10:00:00,1998.490000,18287.000000
std,63.375795,NaN,20.786418,1657.984045


In [5]:
df.describe(include='object')

,Invoice,StockCode,Description,Country
count,32198,32198,32137,32198
unique,1537,2890,2867,23
top,490074,85123A,WHITE HANGING HEART T-LIGHT HOLDER,United Kingdom
freq,580,237,236,30726


## Handling missing values

In [6]:
#counting null values
null_values = df.isnull().mean()*100
null_values = null_values[null_values > 0]

In [7]:
null_percentages_df = null_values.reset_index()
null_percentages_df.columns = ['Column', 'Null_Percentage']

# Plot the null percentages as a bar chart using Plotly
fig = px.bar(null_percentages_df, y='Column', x='Null_Percentage',
             title="Percentage of Null Values per Column",
             labels={'Null_Percentage': 'Percentage (%)', 'Column': 'DataFrame Columns'},
             text_auto='.2f')  # This adds the percentage on top of each bar
fig.show()


### Filling in null descriptions

In [8]:
df[df['Description'].isnull()]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,491639,20668,NaN,-1395,2009-12-11 16:02:00,0.0,NaN,United Kingdom
2,490016,21982,NaN,-1012,2009-12-03 12:30:00,0.0,NaN,United Kingdom
4,489806,18010,NaN,-770,2009-12-02 12:42:00,0.0,NaN,United Kingdom
5,490540,37446,NaN,-725,2009-12-07 09:24:00,0.0,NaN,United Kingdom
9,491108,84922,NaN,-360,2009-12-09 14:29:00,0.0,NaN,United Kingdom
...,...,...,...,...,...,...,...,...
77,489659,21350,NaN,230,2009-12-01 17:39:00,0.0,NaN,United Kingdom
78,490688,21133,NaN,270,2009-12-07 13:50:00,0.0,NaN,United Kingdom
79,490015,21982,NaN,467,2009-12-03 12:29:00,0.0,NaN,United Kingdom
80,489898,79323G,NaN,954,2009-12-03 09:40:00,0.0,NaN,United Kingdom


In [9]:
# Group by 'StockCode' and aggregate unique descriptions and counts
stockcode_analysis = (
    df.groupby('StockCode')['Description']
    .agg(['nunique', 'count', lambda x: x.dropna().unique().tolist()])
    .reset_index()
)

# Rename columns for clarity
stockcode_analysis.columns = ['StockCode', 'Unique_Descriptions_Count', 'Total_Appearance', 'Unique_Descriptions']

# Filter for StockCodes with more than one unique description
stockcode_multiple_descriptions = stockcode_analysis[stockcode_analysis['Unique_Descriptions_Count'] > 1]

print(stockcode_multiple_descriptions)


     StockCode  Unique_Descriptions_Count  Total_Appearance  \
349      21100                          2                35   
374      21133                          2                23   
459      21249                          2                12   
623      21450                          2                 4   
652      21493                          2                29   
841      21733                          2                89   
1145     22139                          2               136   
1198     22195                          2                48   
1229     22232                          2                 6   
1325     22343                          2                 8   
1326     22344                          2                 9   
1327     22345                          2                 9   
1328     22346                          2                 7   
1329     22347                          2                27   
1469     51008                          2              

In [10]:
from collections import Counter
description_map = df.groupby('StockCode')['Description'].apply(lambda x: Counter(x).most_common(1)[0][0]).to_dict()
print(description_map)

{10002: 'INFLATABLE POLITICAL GLOBE ', 10080: 'GROOVY CACTUS INFLATABLE', 10109: 'BENDY COLOUR PENCILS', 10120: 'DOGGY RUBBER', 10125: 'MINI FUNKY DESIGN TAPES', 10133: 'COLOURING PENCILS BROWN TUBE', 10134: 'COLOURING PENCILS BROWN TUBE', 10135: 'COLOURING PENCILS BROWN TUBE', 10138: 'ASSORTED COLOUR JUMBO PEN', 11001: 'ASSTD DESIGN RACING CAR PEN', 15030: 'FAN BLACK FRAME ', 15034: 'PAPER POCKET TRAVELING FAN ', 15036: 'ASSORTED COLOURS SILK FAN', 15039: 'SANDALWOOD FAN', 16011: ' ANIMAL STICKERS', 16012: 'FOOD/DRINK SPUNGE STICKERS', 16014: 'SMALL CHINESE STYLE SCISSOR', 16015: 'MEDIUM CHINESE STYLE SCISSOR', 16016: 'LARGE CHINESE STYLE SCISSOR', 16033: 'MINI HIGHLIGHTER PENS', 16046: 'TEATIME PEN CASE & PENS', 16047: 'POP ART PEN CASE & PENS', 16048: 'TEATIME ROUND PENCIL SHARPENER ', 16049: 'TEATIME GEL PENS ASST', 16051: 'TEATIME PENCIL W STAMP ASST', 16119: 'FUNKY GIRL ADDRESS BOOK', 16212: 'FUNKY GIRLZ ASST MAGNETIC MEMO PAD', 16215: 'FUNKY GIRLZ MAGNETIC TO DO LIST ', 16216: '

In [11]:
df['Actual Description'] = df['StockCode'].map(description_map)

In [12]:
df.isnull().sum()

Invoice                  0
StockCode                0
Description             61
Quantity                 0
InvoiceDate              0
Price                    0
Customer ID           8360
Country                  0
Actual Description      28
dtype: int64

### Missing Customer IDs:

In [13]:
missing_customer_df = df[df['Customer ID'].isnull()]
df['Customer ID'] = df.groupby('Invoice')['Customer ID'].transform(lambda x: x.ffill().bfill())


In [14]:
filled_customer_df = df[df['Customer ID'].isnull()]
print("After filling, remaining rows with missing Customer ID:")
print(filled_customer_df)

After filling, remaining rows with missing Customer ID:
      Invoice StockCode     Description  Quantity         InvoiceDate   Price  \
0      491639     20668             NaN     -1395 2009-12-11 16:02:00    0.00   
1      489660     35956            lost     -1043 2009-12-01 17:43:00    0.00   
2      490016     21982             NaN     -1012 2009-12-03 12:30:00    0.00   
3      489899   79323GR    sold as gold      -954 2009-12-03 09:41:00    0.00   
4      489806     18010             NaN      -770 2009-12-02 12:42:00    0.00   
...       ...       ...             ...       ...                 ...     ...   
32191  491966       DOT  DOTCOM POSTAGE         1 2009-12-14 17:16:00  815.01   
32192  490741       DOT  DOTCOM POSTAGE         1 2009-12-07 17:56:00  862.38   
32193  490074       DOT  DOTCOM POSTAGE         1 2009-12-03 14:39:00  862.67   
32194  491969       DOT  DOTCOM POSTAGE         1 2009-12-14 17:57:00  882.72   
32195  490149       DOT  DOTCOM POSTAGE         1 200

### method 2 

In [15]:
missing_customer_invoices = df[df['Customer ID'].isnull()]

In [16]:
missing_customer_invoices.to_excel('missing_customer_invoices.xlsx', index = False)

In [17]:
invoice_item_counts = missing_customer_invoices.groupby('Invoice').size().reset_index(name='entry_count')

In [18]:
invoice_item_counts.sort_values(by='entry_count', ascending=False, inplace=True)

In [19]:
len(invoice_item_counts)

167

In [20]:
invoice_item_counts.head(20)

,Invoice,entry_count
48,490074,580
149,491966,579
53,490149,559
151,491969,548
82,490741,546
34,489857,516
83,490745,507
7,489597,502
123,491155,500
152,491970,486


In [21]:
invoice_counts = df.groupby('Invoice').size().reset_index(name='count')
invoice_counts.sort_values(by='count', ascending=False, inplace=True)
invoice_counts.head(20)

,Invoice,count
329,490074,580
1234,491966,579
389,490149,559
1237,491969,548
687,490741,546
216,489857,516
688,490745,507
78,489597,502
915,491155,500
1238,491970,486


In [22]:
import plotly.graph_objects as go

# Merge the dataframes to get entry counts and overall counts
merged_df = invoice_counts.merge(invoice_item_counts, on='Invoice', how='left', suffixes=('_overall', '_missing'))

# Mark whether customer ID is missing
merged_df['Customer_ID_Status'] = merged_df['Invoice'].apply(
    lambda x: 'Missing' if x in missing_customer_invoices['Invoice'].values else 'Present'
)

# Sort by frequency and get the top 20 invoices
top_20_invoices = merged_df.sort_values(by='count', ascending=False).head(20)

# Create a bubble chart
fig = go.Figure()
size_scale_factor = 0.2  # You can adjust this factor to make the bubbles smaller or larger
# Add the bubbles for the top 20 invoices
fig.add_trace(go.Scatter(
    x=top_20_invoices['Invoice'],  # Invoice IDs on x-axis
    y=top_20_invoices['count'],  # Invoice frequency on y-axis
    mode='markers',  # Bubble chart mode
    marker=dict(
        size=top_20_invoices['count']*size_scale_factor,  # Bubble size corresponds to invoice count
        color=top_20_invoices['Customer_ID_Status'].map({'Missing': 'red', 'Present': 'blue'}),  # Color by Missing or Present status
        opacity=0.6,
        line=dict(width=2)
    ),
    text=top_20_invoices['Customer_ID_Status'],  # Tooltip text for each bubble
    hovertemplate='%{text}<br>Invoice ID: %{x}<br>Count: %{y}<extra></extra>',  # Tooltip content
))

# Layout settings for the plot
fig.update_layout(
    title='Top 20 Invoices as Bubbles with Missing Customer IDs Highlighted',
    xaxis_title='Invoice ID',
    yaxis_title='Invoice Count',
    template='plotly_dark',  # Set the theme
    showlegend=False,  # Hide the legend
    height= 700,
    width = 900
)

fig.show()


## Observations:
### 15 of the top 20 invoice entries have missing customer IDs. This could mean different things

In [23]:
df['InvoiceDate'].head(10)

0   2009-12-11 16:02:00
1   2009-12-01 17:43:00
2   2009-12-03 12:30:00
3   2009-12-03 09:41:00
4   2009-12-02 12:42:00
5   2009-12-07 09:24:00
6   2009-12-03 12:09:00
7   2009-12-02 13:23:00
8   2009-12-03 18:28:00
9   2009-12-09 14:29:00
Name: InvoiceDate, dtype: datetime64[ns]

In [24]:
df['InvoiceDate1'] = pd.to_datetime(df['InvoiceDate'], format='%d/%m/%Y %H:%M', errors='coerce')

In [25]:
df = df.drop('InvoiceDate', axis=1)

In [26]:
df.rename(columns={'InvoiceDate1': 'InvoiceDate'}, inplace=True)

In [27]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'Price',
       'Customer ID', 'Country', 'Actual Description', 'InvoiceDate'],
      dtype='object')

In [28]:
df['InvoiceDate_Date'] = df['InvoiceDate'].dt.date
daily_invoice_counts = df.groupby('InvoiceDate_Date')['Invoice'].nunique().reset_index(name='UniqueInvoices')

In [29]:
daily_invoice_counts['UniqueInvoices'].sum()

1537

In [30]:
df.isnull().sum()

Invoice                  0
StockCode                0
Description             61
Quantity                 0
Price                    0
Customer ID           8360
Country                  0
Actual Description      28
InvoiceDate              0
InvoiceDate_Date         0
dtype: int64

In [31]:
missing_customer_invoices = df[df['Customer ID'].isnull()]
daily_missing_counts = missing_customer_invoices.groupby('InvoiceDate_Date')['Invoice'].nunique().reset_index(name='MissingInvoices')

In [32]:
daily_counts = pd.merge(daily_invoice_counts, daily_missing_counts, on='InvoiceDate_Date', how='left').fillna(0)

# Plot the data with Plotly
fig = px.line(
    daily_counts,
    x='InvoiceDate_Date',
    y=['UniqueInvoices', 'MissingInvoices'],
    title="Unique Invoices and Missing Customer IDs per Day",
    labels={'InvoiceDate_Date': 'Date', 'value': 'Count', 'variable': 'Invoice Type'},
    markers=True,  # Add markers to make points more visible
    height=600
)

# Show the plot
fig.update_layout(xaxis_tickangle=45)  # Rotate x-axis labels for better readability
fig.show()

### As these missing customer IDs hold 25% of the data and involve large invoices, it is worth leaving them as part of analysis and fininding new ways of filling the null values:

#### Finding new placeholders for each unique invoice

In [33]:
# Step 1: Identify invoices with missing customer IDs
missing_customer_invoices = df[df['Customer ID'].isnull()]

# Step 2: Create a unique identifier for each invoice with missing Customer ID
# We generate sequential placeholders starting from M12345
# Use the invoice number as a key to ensure each unique invoice gets the same placeholder

# Create a dictionary that maps unique invoices to a placeholder ID
unique_invoices = missing_customer_invoices['Invoice'].unique()
placeholder_dict = {invoice: f"M{12345 + i}" for i, invoice in enumerate(unique_invoices)}

# Step 3: Replace missing customer IDs with the placeholders
df['Customer ID'] = df.apply(
    lambda row: placeholder_dict.get(row['Invoice'], row['Customer ID']) if pd.isnull(row['Customer ID']) else row['Customer ID'],
    axis=1
)

# Check the result to verify
print(df.head())


  Invoice StockCode   Description  Quantity  Price Customer ID  \
0  491639     20668           NaN     -1395    0.0      M12345   
1  489660     35956          lost     -1043    0.0      M12346   
2  490016     21982           NaN     -1012    0.0      M12347   
3  489899   79323GR  sold as gold      -954    0.0      M12348   
4  489806     18010           NaN      -770    0.0      M12349   

          Country                Actual Description         InvoiceDate  \
0  United Kingdom   DISCO BALL CHRISTMAS DECORATION 2009-12-11 16:02:00   
1  United Kingdom                              lost 2009-12-01 17:43:00   
2  United Kingdom                               NaN 2009-12-03 12:30:00   
3  United Kingdom               GREEN CHERRY LIGHTS 2009-12-03 09:41:00   
4  United Kingdom  ASSORTED FRAGRANCE BATH CONFETTI 2009-12-02 12:42:00   

  InvoiceDate_Date  
0       2009-12-11  
1       2009-12-01  
2       2009-12-03  
3       2009-12-03  
4       2009-12-02  


In [34]:

# Step 1: Identify invoices with missing customer IDs
missing_customer_invoices = df[df['Customer ID'].isnull()]

# Step 2: Create a unique identifier for each invoice with missing Customer ID
# We generate sequential placeholders starting from M12345
# Use the invoice number as a key to ensure each unique invoice gets the same placeholder

# Create a dictionary that maps unique invoices to a placeholder ID
unique_invoices = missing_customer_invoices['Invoice'].unique()
placeholder_dict = {invoice: f"M{12345 + i}" for i, invoice in enumerate(unique_invoices)}

# Step 3: Replace missing customer IDs with the placeholders
df['Customer ID'] = df.apply(lambda row: placeholder_dict.get(row['Invoice'], row['Customer ID']), axis=1)

# Check the result to verify
df.head()


,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country,Actual Description,InvoiceDate,InvoiceDate_Date
0,491639,20668,NaN,-1395,0.0,M12345,United Kingdom,DISCO BALL CHRISTMAS DECORATION,2009-12-11 16:02:00,2009-12-11
1,489660,35956,lost,-1043,0.0,M12346,United Kingdom,lost,2009-12-01 17:43:00,2009-12-01
2,490016,21982,NaN,-1012,0.0,M12347,United Kingdom,NaN,2009-12-03 12:30:00,2009-12-03
3,489899,79323GR,sold as gold,-954,0.0,M12348,United Kingdom,GREEN CHERRY LIGHTS,2009-12-03 09:41:00,2009-12-03
4,489806,18010,NaN,-770,0.0,M12349,United Kingdom,ASSORTED FRAGRANCE BATH CONFETTI,2009-12-02 12:42:00,2009-12-02


In [32]:
df.isnull().sum()

Invoice                0
StockCode              0
Description           61
Quantity               0
Price                  0
Customer ID            0
Country                0
Actual Description    28
InvoiceDate            0
InvoiceDate_Date       0
dtype: int64

In [35]:
a = df[df['Customer ID'].notna() & df['Customer ID'].str.startswith('M1')]
len(a)

8360

In [36]:
type(a['Invoice'])

pandas.core.series.Series

In [37]:
duplicate_rows = df[df.duplicated(keep=False)]

# Sorting the data by certain columns to see the duplicate rows next to each other
duplicate_rows_sorted = duplicate_rows.sort_values(by=['Invoice', 'StockCode', 'Description', 'Customer ID', 'Quantity'])

# Displaying the first 10 records
duplicate_rows_sorted.head(50)

,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country,Actual Description,InvoiceDate,InvoiceDate_Date
12364,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,1.95,16329.0,United Kingdom,SET OF THREE VINTAGE GIFT WRAPS,2009-12-01 11:34:00,2009-12-01
12365,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,1.95,16329.0,United Kingdom,SET OF THREE VINTAGE GIFT WRAPS,2009-12-01 11:34:00,2009-12-01
20976,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,3.75,16329.0,United Kingdom,GLITTER STAR GARLAND WITH BELLS,2009-12-01 11:34:00,2009-12-01
20977,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,3.75,16329.0,United Kingdom,GLITTER STAR GARLAND WITH BELLS,2009-12-01 11:34:00,2009-12-01
21208,489517,21912,VINTAGE SNAKES & LADDERS,1,3.75,16329.0,United Kingdom,VINTAGE SNAKES & LADDERS,2009-12-01 11:34:00,2009-12-01
21209,489517,21912,VINTAGE SNAKES & LADDERS,1,3.75,16329.0,United Kingdom,VINTAGE SNAKES & LADDERS,2009-12-01 11:34:00,2009-12-01
21210,489517,21912,VINTAGE SNAKES & LADDERS,1,3.75,16329.0,United Kingdom,VINTAGE SNAKES & LADDERS,2009-12-01 11:34:00,2009-12-01
21202,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,3.75,16329.0,United Kingdom,VINTAGE SEASIDE JIGSAW PUZZLES,2009-12-01 11:34:00,2009-12-01
21203,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,3.75,16329.0,United Kingdom,VINTAGE SEASIDE JIGSAW PUZZLES,2009-12-01 11:34:00,2009-12-01
3623,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,0.85,16329.0,United Kingdom,PARTY CONE CHRISTMAS DECORATION,2009-12-01 11:34:00,2009-12-01


## Returns analysis

In [38]:
#rows with negative quantities indicate returns:
returns = df[df['Quantity'] < 0]
len(returns)


825

In [40]:
type(df['Invoice'])

pandas.core.series.Series

In [41]:
returns['Return_Type'] = returns.apply(
    lambda row: 'Cancellation-Based' if row['Quantity'] < 0 and str(row['Invoice']).startswith('C') and row['Price'] != 0 
                else 'Zero-Priced', 
    axis=1
)


/var/folders/35/yy_kphpd6c938fbpf2fk1n3r0000gn/T/ipykernel_40778/982180699.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [42]:
# Check the distribution of return types
return_counts = returns['Return_Type'].value_counts()
print(return_counts)

Return_Type
Cancellation-Based    771
Zero-Priced            54
Name: count, dtype: int64


In [43]:
total_transactions = len(df)
total_returns = len(returns)
return_rate = (total_returns / total_transactions) * 100
print(f"Overall Return Rate: {return_rate:.2f}%")

Overall Return Rate: 2.56%


### Analyze Patterns in Returns by Segment

In [44]:
# Calculate return rate by product
product_returns = returns.groupby('StockCode').size()
total_by_product = df.groupby('StockCode').size()
product_return_rate = (product_returns / total_by_product).sort_values(ascending=False).dropna()

# Show top 10 products by return rate
print(product_return_rate.tail(20))


StockCode
22109     0.017544
21888     0.017241
20754     0.016949
22348     0.016393
22189     0.015385
21755     0.014925
21791     0.013889
21754     0.013889
22111     0.012270
84946     0.012195
21491     0.012195
82482     0.012048
84879     0.011765
79323W    0.011628
85014B    0.011364
22113     0.011236
21479     0.010204
21485     0.009174
22114     0.007246
22086     0.006897
dtype: float64


### Returns by day

In [45]:
# Add a daily date column for time-based analysis
returns['InvoiceDate'] = pd.to_datetime(returns['InvoiceDate'], errors='coerce')
returns['ReturnDate'] = returns['InvoiceDate'].dt.date  # Extract only the date part

# Group by the daily date to get daily return counts
daily_returns = returns.groupby('ReturnDate').size()

# Plot daily return trends
import plotly.express as px
fig = px.line(
    daily_returns, 
    x=daily_returns.index.astype(str), 
    y=daily_returns.values, 
    labels={'x': 'Date', 'y': 'Return Count'},
    title="Daily Return Trends"
)
fig.show()


/var/folders/35/yy_kphpd6c938fbpf2fk1n3r0000gn/T/ipykernel_40778/3369936512.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/35/yy_kphpd6c938fbpf2fk1n3r0000gn/T/ipykernel_40778/3369936512.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [49]:
december_8_data = returns[(returns['InvoiceDate'].dt.date == pd.to_datetime('2009-12-08').date()) & (returns['Quantity'] < 0)]

# Extract the hour from 'InvoiceDate' for hourly grouping
december_8_data['Hour'] = december_8_data['InvoiceDate'].dt.hour

# Group by hour and count the number of returns
hourly_returns = december_8_data.groupby('Hour').size()

# Plot the hourly return trend for December 8
import plotly.express as px
fig = px.line(
    hourly_returns, 
    x=hourly_returns.index, 
    y=hourly_returns.values, 
    labels={'x': 'Hour of Day', 'y': 'Return Count'},
    title="Hourly Return Trend for December 8"
)
fig.update_xaxes(dtick=1)  # Show each hour on the x-axis
fig.show()

/var/folders/35/yy_kphpd6c938fbpf2fk1n3r0000gn/T/ipykernel_986/382851870.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
import pandas as pd
import plotly.express as px


returns['Hour'] = returns['InvoiceDate'].dt.hour

# Group by both 'ReturnDate' and 'Hour' and count the returns
hourly_returns = returns.groupby(['ReturnDate', 'Hour']).size().reset_index(name='ReturnCount')

# Plot the hourly return trends for each day
fig = px.line(
    hourly_returns, 
    x='Hour', 
    y='ReturnCount', 
    color='ReturnDate',
    labels={'Hour': 'Hour of Day', 'ReturnCount': 'Return Count', 'ReturnDate': 'Date'},
    title="Hourly Return Trends by Date"
)
fig.update_xaxes(dtick=1)  # Show each hour on the x-axis
fig.show()


/var/folders/35/yy_kphpd6c938fbpf2fk1n3r0000gn/T/ipykernel_40778/275120886.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Return rate by country

In [47]:


# Calculate total transactions per country (items sold or total invoices)
total_transactions_by_country = df.groupby('Country').size()

# Calculate total returns per country
total_returns_by_country = returns.groupby('Country').size()

# Calculate return rate by country (as percentage)
return_rate_by_country = (total_returns_by_country / total_transactions_by_country) * 100

# Show the return rates
print(return_rate_by_country)

Country
Australia                59.090909
Austria                        NaN
Belgium                        NaN
Channel Islands           9.302326
Cyprus                    2.912621
Denmark                  66.666667
EIRE                      6.324111
Finland                        NaN
France                   19.148936
Germany                   6.017192
Greece                         NaN
Italy                   100.000000
Japan                   100.000000
Netherlands                    NaN
Nigeria                 100.000000
Norway                         NaN
Poland                         NaN
Portugal                       NaN
Spain                    11.111111
Sweden                   33.333333
USA                            NaN
United Arab Emirates      1.754386
United Kingdom            2.268437
dtype: float64


In [48]:
import plotly.express as px
import pandas as pd

# Assuming you already have the following variables calculated
# total_transactions_by_country, total_returns_by_country, return_rate_by_country

# Convert the return_rate_by_country into a DataFrame for easier plotting
return_rate_df = pd.DataFrame({
    'Country': return_rate_by_country.index,
    'Return Rate (%)': return_rate_by_country.values
})

# Create a choropleth map
fig = px.choropleth(return_rate_df, 
                    locations='Country', 
                    locationmode='country names',  # Use country names for matching
                    color='Return Rate (%)',  # The values we want to show on the map
                    hover_name='Country',  # Information to display on hover
                    color_continuous_scale='reds',  # You can choose different color scales
                    labels={'Return Rate (%)': 'Return Rate (%)'},  # Label for the color scale
                    title='Return Rate by Country')

fig.update_layout(
      # Set the paper background color to red
    title_x=0.5,  # Center the title
    title_y=0.95  # Adjust title positioning (optional)
)
# Show the map
fig.show()


In [ ]:
# Sort return rates in descending order
sorted_return_rate_by_country = return_rate_by_country.sort_values(ascending=False)

# Plot the return rates by country using Plotly
import plotly.express as px
fig = px.bar(
    sorted_return_rate_by_country, 
    labels={'index': 'Country', 'value': 'Return Rate (%)'},
    title="Return Rate per Country",
    color_discrete_sequence=['red']  # Set the bar color to red
)

fig.update_layout(
    plot_bgcolor='white',  # Set the plot background color (optional
    title_x=0.5,           # Center the title
    title_y=0.95           # Adjust title positioning (optional)
)

fig.show()


### Investigate Return Drivers

In [ ]:
# Separate purchases and cancellations
purchases = df[~df['Invoice'].str.startswith('C')]
cancellations = df[df['Invoice'].str.startswith('C')]

# Match purchases and cancellations based on the Invoice number (remove 'C' from cancellation invoice)
cancellations['Invoice'] = cancellations['Invoice'].str[1:]

# Merge purchases and cancellations on the 'Invoice' column
merged_df = pd.merge(purchases, cancellations, on='Invoice')

# Calculate the time difference between purchase and cancellation in minutes
merged_df['TimeDifference'] = (merged_df['InvoiceDate_cancel'] - merged_df['InvoiceDate_purchase']).dt.total_seconds() / 60

# Plotting using Plotly
fig = px.histogram(merged_df, 
                   x='TimeDifference', 
                   nbins=20, 
                   title="Distribution of Time Differences Between Purchases and Cancellations",
                   labels={'TimeDifference': 'Time Difference (Minutes)'},
                   template='plotly_dark')

# Show the plot
fig.show()

# Optionally print the merged dataframe to see the results
print(merged_df[['Invoice', 'Description_purchase', 'TimeDifference']])

In [303]:
# Separate purchases and cancellations
purchases = df[~df['Invoice'].str.startswith('C')]
cancellations = df[df['Invoice'].str.startswith('C')]

In [305]:
print("Purchases invoices:", purchases['Invoice'].unique())
print("Cancellations invoices (before stripping):", cancellations['Invoice'].unique())

Purchases invoices: ['489824' '491758' '491047' ... '490091' '490007' '490932']
Cancellations invoices (before stripping): ['C491080' 'C491411' 'C490807' 'C489528' 'C490946' 'C490744' 'C490743'
 'C491728' 'C491166' 'C491693' 'C490358' 'C490773' 'C491723' 'C491040'
 'C490923' 'C490148' 'C491003' 'C489570' 'C490980' 'C490076' 'C490942'
 'C491416' 'C491414' 'C491676' 'C490973' 'C489449' 'C490931' 'C491546'
 'C491611' 'C490700' 'C490283' 'C491015' 'C490746' 'C490987' 'C491099'
 'C491002' 'C490935' 'C490996' 'C491409' 'C491190' 'C491594' 'C491417'
 'C491004' 'C489527' 'C490460' 'C490120' 'C490702' 'C491705' 'C491466'
 'C490480' 'C489643' 'C489541' 'C490866' 'C490510' 'C491818' 'C489858'
 'C490701' 'C490994' 'C491461' 'C491460' 'C489636' 'C491013' 'C491410'
 'C490296' 'C489459' 'C490122' 'C491985' 'C491260' 'C491084' 'C490798'
 'C489955' 'C489685' 'C491632' 'C491020' 'C491521' 'C490947' 'C489713'
 'C491598' 'C490129' 'C490309' 'C491610' 'C489881' 'C490544' 'C491827'
 'C489583' 'C491402' 'C49

In [310]:
cancellations_mapping = cancellations.set_index(['Invoice', 'Description'])['InvoiceDate'].to_dict()

In [311]:
cancellations_mapping

{('491080', 'GOLD EAR MUFF HEADPHONES'): Timestamp('2009-12-09 13:07:00'),
 ('491411',
  'CAKE STAND WHITE TWO TIER LACE'): Timestamp('2009-12-10 18:58:00'),
 ('490807', 'BIG DOUGHNUT FRIDGE MAGNETS'): Timestamp('2009-12-08 12:25:00'),
 ('489528',
  '15CM CHRISTMAS GLASS BALL 20 LIGHTS'): Timestamp('2009-12-01 11:50:00'),
 ('490946', 'LADS ONLY TISSUE BOX'): Timestamp('2009-12-08 14:28:00'),
 ('490744',
  'RED/WHITE DOTS RUFFLED UMBRELLA'): Timestamp('2009-12-07 18:00:00'),
 ('490743',
  'CERAMIC CAKE STAND + HANGING CAKES'): Timestamp('2009-12-07 17:59:00'),
 ('491728', 'COFFEE MUG CAT + BIRD DESIGN'): Timestamp('2009-12-14 09:18:00'),
 ('491166', 'MOBILE VINTAGE HEARTS '): Timestamp('2009-12-10 11:00:00'),
 ('491693',
  'SET/3 OCEAN SCENT CANDLE JEWEL BOX'): Timestamp('2009-12-13 13:22:00'),
 ('490358', 'CREAM SWEETHEART MINI CHEST'): Timestamp('2009-12-04 16:51:00'),
 ('490773',
  'LARGE CAKE STAND  HANGING STRAWBERY'): Timestamp('2009-12-08 11:19:00'),
 ('491723',
  'WHITE HANGING 

In [312]:
purchases['CancellationDate'] = purchases.apply(
    lambda row: cancellations_mapping.get((row['Invoice'], row['Description']), pd.NaT),
    axis=1
)
purchases

/var/folders/35/yy_kphpd6c938fbpf2fk1n3r0000gn/T/ipykernel_46773/18238018.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country,Actual Description,InvoiceDate,InvoiceDate_Date,CancellationDate
0,489824,20914,SET/5 RED SPOTTY LID GLASS BOWLS,1,2.95,16797.0,United Kingdom,SET/5 RED SPOTTY LID GLASS BOWLS,2009-12-02 13:33:00,2009-12-02,NaT
1,491758,84946,ANTIQUE SILVER TEA GLASS ETCHED,6,1.25,17908.0,United Kingdom,ANTIQUE SILVER TEA GLASS ETCHED,2009-12-14 12:24:00,2009-12-14,NaT
2,491047,21982,NaN,-50,0.00,M12345,United Kingdom,NaN,2009-12-09 10:04:00,2009-12-09,NaT
3,490149,21396,RED SPOTTY EGG CUP,2,3.43,M12346,United Kingdom,RED SPOTTY EGG CUP,2009-12-04 09:43:00,2009-12-04,NaT
4,489916,22083,PAPER CHAIN KIT RETRO SPOT,12,2.95,15194.0,United Kingdom,PAPER CHAIN KIT RETRO SPOT,2009-12-03 10:37:00,2009-12-03,NaT
...,...,...,...,...,...,...,...,...,...,...,...
32193,490083,21490,SET OF THREE 50'S GIFT WRAPS,10,1.95,14031.0,United Kingdom,SET OF THREE 50'S GIFT WRAPS,2009-12-03 15:35:00,2009-12-03,NaT
32194,491584,20972,PINK CREAM FELT CRAFT TRINKET BOX,4,1.25,12854.0,United Kingdom,PINK CREAM FELT CRAFT TRINKET BOX,2009-12-11 12:55:00,2009-12-11,NaT
32195,489830,90214A,"LETTER ""A"" BLING KEY RING",1,1.25,13623.0,United Kingdom,"LETTER ""A"" BLING KEY RING",2009-12-02 13:57:00,2009-12-02,NaT
32196,489856,84081,WHITE CERAMIC PARTY SET (5PCS),1,8.65,M12356,United Kingdom,WHITE CERAMIC PARTY SET (5PCS),2009-12-02 14:36:00,2009-12-02,NaT


In [306]:
cancellations['Invoice'] = cancellations['Invoice'].str[1:]

# Debug: Check if invoice numbers are now aligned
print("Cancellations invoices (after stripping):", cancellations['Invoice'].unique())

Cancellations invoices (after stripping): ['491080' '491411' '490807' '489528' '490946' '490744' '490743' '491728'
 '491166' '491693' '490358' '490773' '491723' '491040' '490923' '490148'
 '491003' '489570' '490980' '490076' '490942' '491416' '491414' '491676'
 '490973' '489449' '490931' '491546' '491611' '490700' '490283' '491015'
 '490746' '490987' '491099' '491002' '490935' '490996' '491409' '491190'
 '491594' '491417' '491004' '489527' '490460' '490120' '490702' '491705'
 '491466' '490480' '489643' '489541' '490866' '490510' '491818' '489858'
 '490701' '490994' '491461' '491460' '489636' '491013' '491410' '490296'
 '489459' '490122' '491985' '491260' '491084' '490798' '489955' '489685'
 '491632' '491020' '491521' '490947' '489713' '491598' '490129' '490309'
 '491610' '489881' '490544' '491827' '489583' '491402' '491544' '491029'
 '490991' '491441' '491819' '491130' '489595' '491469' '491459' '491614'
 '491621' '490929' '491017' '490520' '491593' '490565' '490348' '491413'
 '489534'

/var/folders/35/yy_kphpd6c938fbpf2fk1n3r0000gn/T/ipykernel_46773/492893453.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [308]:
# Merge purchases and cancellations on both 'Invoice' and 'Description'
merged_df = pd.merge(purchases, cancellations, on=['Invoice', 'Description'], suffixes=('_purchase', '_cancel'), how='inner')
merged_df

,Invoice,StockCode_purchase,Description,Quantity_purchase,Price_purchase,Customer ID_purchase,Country_purchase,Actual Description_purchase,InvoiceDate_purchase,InvoiceDate_Date_purchase,StockCode_cancel,Quantity_cancel,Price_cancel,Customer ID_cancel,Country_cancel,Actual Description_cancel,InvoiceDate_cancel,InvoiceDate_Date_cancel


In [300]:
cancellations

,Invoice,StockCode,Description,Quantity,Price,Customer ID,Country,Actual Description,InvoiceDate,InvoiceDate_Date
60,491080,20781,GOLD EAR MUFF HEADPHONES,-3,5.49,14593.0,United Kingdom,GOLD EAR MUFF HEADPHONES,2009-12-09 13:07:00,2009-12-09
77,491411,22215,CAKE STAND WHITE TWO TIER LACE,-2,8.50,15309.0,United Kingdom,CAKE STAND WHITE TWO TIER LACE,2009-12-10 18:58:00,2009-12-10
79,490807,21700,BIG DOUGHNUT FRIDGE MAGNETS,-24,0.85,13091.0,United Kingdom,BIG DOUGHNUT FRIDGE MAGNETS,2009-12-08 12:25:00,2009-12-08
87,489528,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,-1,7.95,16714.0,United Kingdom,15CM CHRISTMAS GLASS BALL 20 LIGHTS,2009-12-01 11:50:00,2009-12-01
129,490946,22095,LADS ONLY TISSUE BOX,-36,1.06,13091.0,United Kingdom,LADS ONLY TISSUE BOX,2009-12-08 14:28:00,2009-12-08
...,...,...,...,...,...,...,...,...,...,...
31751,489911,21432,SET OF 3 CASES WOODLAND DESIGN,-1,5.95,12705.0,Germany,SET OF 3 CASES WOODLAND DESIGN,2009-12-03 10:26:00,2009-12-03
31781,491156,21843,RETRO SPOT CAKE STAND,-2,9.95,15523.0,United Kingdom,RETRO SPOT CAKE STAND,2009-12-10 09:54:00,2009-12-10
31829,489459,90185B,AMETHYST DIAMANTE EXPANDABLE RING,-3,4.25,17592.0,United Kingdom,AMETHYST DIAMANTE EXPANDABLE RING,2009-12-01 10:44:00,2009-12-01
31846,491460,21843,RETRO SPOT CAKE STAND,-5,10.95,15162.0,United Kingdom,RETRO SPOT CAKE STAND,2009-12-11 11:31:00,2009-12-11
